# Système linéaire implicite et explicite Max-Plus


Tout comme la boîte à outils Scilab, une structure Max-Plus pour Julia est proposée afin de manipuler des systèmes dynamiques linéaires implicites sous la forme suivante :

$$\left \{ \begin{array}{l}
X(n) &= D X(n) \oplus A X(n-1) \oplus B U(n) \\
Y(n) &= C X(n)
\end{array} \right.$$

Avec $x(0) = x_0$ connu. Le quintuplé $(A,B,C,D,x_0)$ est manipulé par la structure `MPSysLin`. Ce document fait suite à [l'introduction de la boîte à outil Julia pour l'algèbre Max-Plus](https://nbviewer.jupyter.org/github/Lecrapouille/MaxPlus.jl/blob/master/tutorial/core-fr.ipynb) et nous verrons comment manipuler cette structure qui sera utilisée sur un problème concret dans le [document suivant](https://nbviewer.jupyter.org/github/Lecrapouille/MaxPlus.jl/blob/master/tutorial/flowshop-fr.ipynb).

# Prérequis

Si ce n'est pas déjà fait, chargeons la boîte à outils Max-Plus. Pour rappel, le document précédent explique plus longuement comment faire. 

In [10]:
push!(LOAD_PATH, pwd())
using MaxPlus, SparseArrays

Toujours pour des soucis d'appel à fonction `Base.show` depuis un document Jupyter, nous forçons l'affichage en mode plein :

In [3]:
Base.show(io::IO, ::MIME"text/latex", x::MP) = show(io, MIME"text/plain", x)
Base.show(io::IO, ::MIME"text/latex", A::ArrMP) = show(io, MIME"text/plain", A)
Base.show(io::IO, ::MIME"text/latex", S::SpaMP) = show(io, MIME"text/plain", S)
Base.show(io::IO, ::MIME"text/latex", S::MPSysLin) = show(io, MIME"text/plain", S)

Pour ce document Jupyter, on va se créer une fonction qui forcera l'affichage LaTeX :

In [4]:
using LaTeXStrings

latexify(S1, name="S") = latexstring(name, "=", LaTeX(S1))

latexify (generic function with 2 methods)

## Construction d'un système linéaire implicite Max-Plus

Créons notre premier système linéaire Max-Plus implicite grâce aux constructeurs de la structure Julia associés `MPSysLin(A, B, C [,D, x0])` :

$$S_1=\left\{\begin{array}{lcl}
x_n & = & \left[
\begin{array}{*{20}c}
0 & . \\
. & 0 \\
\end{array}
\right]
 x_n \oplus \left[
\begin{array}{*{20}c}
1 & 2 \\
3 & 4 \\
\end{array}
\right]
 x_{n-1} \oplus \left[
\begin{array}{*{20}c}
0 \\
0 \\
\end{array}
\right]
 u_n\\y_n & = & \left[
\begin{array}{*{20}c}
0 & 0 \\
\end{array}
\right]
 x_n\\x_0 & = & \left[
\begin{array}{*{20}c}
. \\
. \\
\end{array}
\right]
\end{array}\right.$$

La matrice $D$ et le vecteur $x_0$ sont optionnels. S'ils ne sont pas donnés ils seront automatiquement créés et remplis de $\varepsilon$.

In [11]:
S1 = MPSysLin(MP([1 2; 3 4]),      # A
              MP([0;0]),           # B
              MP([0 0]),           # C
              eye(MP,2,2),         # D (optionel)
              spzeros(MP,2,1))     # x0 (optionel)

Implicit dynamic linear Max-Plus system:
  x(n) = D*x(n) + A*x(n-1) + B*u(n)
  y(n) = C*x(n)
  x(0) = x0

with:
D = 2×2 Max-Plus dense matrix:
  0   .
  .   0

A = 2×2 Max-Plus dense matrix:
  1   2
  3   4

B = 2-element Max-Plus vector:
  0
  0

C = 1×2 Max-Plus dense matrix:
  0   0

x0 = 2×1 Max-Plus sparse matrix with 0 stored entries

Et le deuxième système implicite :

$$S_2=\left\{\begin{array}{lcl}
x_n & = & \left[
\begin{array}{*{20}c}
0 & . & . \\
. & 0 & . \\
. & . & 0 \\
\end{array}
\right]
 x_n \oplus \left[
\begin{array}{*{20}c}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9 \\
\end{array}
\right]
 x_{n-1} \oplus \left[
\begin{array}{*{20}c}
0 \\
0 \\
0 \\
\end{array}
\right]
 u_n\\y_n & = & \left[
\begin{array}{*{20}c}
0 & 0 & 0 \\
\end{array}
\right]
 x_n\\x_0 & = & \left[
\begin{array}{*{20}c}
. \\
. \\
. \\
\end{array}
\right]
\end{array}\right.$$

In [12]:
S2 = MPSysLin(MP([1 2 3; 4 5 6; 7 8 9]),  # A
              MP([0;0;0]),                # B
              MP([0 0 0]),                # C
              eye(MP,3,3))                # D
                                          # x0 to zeros vector

LoadError: UndefVarError: mpeye not defined

## Affichage de systèmes implicites Max-Plus en $\LaTeX$

Comme cet affichage n'est pas le plus amicale, on peut générer du code $\LaTeX$ soit avec la fonction `LaTeX` (et si l'on ne passe pas l'argument `IO` alors une chaîne de caractères sera retournée) soit via la fonction `Base.show` en passant l'argument `::MIME"text/latex"` :

In [ ]:
LaTeX(stdout, S1)  # show(stdout, "text/latex", S1)

## Composition de systèmes implicites Max-Plus

Tout comme Scilab, Julia permet de surcharger des opérateurs de base, que l'on utilisera pour composer des systèmes linéaires.

#### Composition parallèle

In [ ]:
S = S1 + S2; latexify(S)

#### Composition diagonale

In [ ]:
S = S1 | S2; latexify(S)

#### Composition en série

In [ ]:
S = S1 * S2; latexify(S)

#### Entrées communes

In [ ]:
S = [S1 S2]; latexify(S)

#### Sorties communes

In [ ]:
S = [S1; S2]; latexify(S)

#### Composition avec retro-action

In [ ]:
S = S1 / S2; latexify(S)

## Conversion système implicte Max-Plus vers système explicite Max-Plus

In [ ]:
# TODO S = mpexplicit(S1); latexify(S)

## Simulation de système linéaire Max-Plus

La fonction `mpsimul` permet de retourner les états $X(n)$ (ou bien le dernier état) d'un système linéaire Max-Plus en injectant des données $U(n)$ Max-Plus sous la forme d'un vecteur.

In [ ]:
u = Vector(MP(1:0.5:5))
y = mpsimul(S, u, true) # Retourne l'historiques des états

In [ ]:
mpsimul(S, u, false) # Retourne le dernier état calculé

On peut ploter le résultat directement les nombres Max-Plus.

In [ ]:
using Plots

In [ ]:
plot(u, y, label=["simulation"], legend=:topleft)

Autre exemple, système avec des entrées communes :

In [ ]:
u = [MP(1:0.5:5) MP(1:0.5:5)]
y = mpsimul([S1 S2], u, true)